<a href="https://colab.research.google.com/github/Shivam-7800/Diffusion-models/blob/main/diffusion_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install torch torchvision


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [4]:
from PIL import Image
import numpy as np
import torch
import torchvision.transforms as transforms

# Load the image
image_path = '/content/drive/MyDrive/F.jpeg'
image = Image.open(image_path).convert('RGB')

# Define the transformation to convert the image to a tensor
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Resize to match model input size
    transforms.ToTensor(),          # Convert to tensor
])
image_tensor = transform(image).unsqueeze(0)  # Add batch dimension


In [5]:
class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()
        # Define the UNet layers
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            # Add more layers
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64, 3, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            # Add more layers
        )

    def forward(self, x):
        # Define the forward pass
        enc = self.encoder(x)
        dec = self.decoder(enc)
        return dec

# Instantiate the model
model = UNet()


In [6]:
def forward_diffusion(x, t, noise_schedule):
    noise = torch.randn_like(x)
    noisy_x = (1 - noise_schedule[t]) * x + noise_schedule[t] * noise
    return noisy_x

# Simulate a simple noise schedule
noise_schedule = np.linspace(0, 1, 100)

# Apply forward diffusion
time_step = 50  # Example time step
noisy_image_tensor = forward_diffusion(image_tensor, time_step, noise_schedule)


In [7]:
def forward_diffusion(x, t, noise_schedule):
    noise = torch.randn_like(x)
    noisy_x = (1 - noise_schedule[t]) * x + noise_schedule[t] * noise
    return noisy_x

# Simulate a simple noise schedule
noise_schedule = np.linspace(0, 1, 100)

# Apply forward diffusion
time_step = 50  # Example time step
noisy_image_tensor = forward_diffusion(image_tensor, time_step, noise_schedule)


In [8]:
def reverse_denoising(noisy_x, t, model, noise_schedule):
    with torch.no_grad():
        predicted_noise = model(noisy_x)
        denoised_x = (noisy_x - noise_schedule[t] * predicted_noise) / (1 - noise_schedule[t])
    return denoised_x

# Apply reverse denoising using the pre-trained model
denoised_image_tensor = reverse_denoising(noisy_image_tensor, time_step, model, noise_schedule)


In [9]:
def tensor_to_image(tensor):
    tensor = tensor.squeeze(0)  # Remove batch dimension
    tensor = tensor.clamp(0, 1)  # Clamp values to [0, 1]
    image = transforms.ToPILImage()(tensor)
    return image

# Convert the denoised tensor back to an image
denoised_image = tensor_to_image(denoised_image_tensor)
denoised_image.save('denoised_image.jpg')
denoised_image.show()
